I am using the essemble method to classify the mushrooms given the data set. I will be using a selection of Bagging and Boosting methods and each method will be tested using a 10 fold validation. 

Do vote for this if you like this analysis :)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import time

## Exploratory analysis and data pre-processing

Load the dataset and do some quick exploratory analysis. It seems that many of the entries are encoded in alphabets. To make the data useful for machine learning, we need to convert them into integers.

In [ ]:
data = pd.read_csv('../input/mushrooms.csv', index_col=False)
data.head(5)

In [ ]:
print(data.shape)

In [ ]:
data.describe()

In [ ]:
encoder = LabelEncoder()

for col in data.columns:
    data[col] = encoder.fit_transform(data[col])
 
data.head()

Let's take a look at the number of Poisonous (1) and Edible (0) cases from the dataset. From the output shown below, majority of the cases are Edible (0).

In [ ]:
print(data.groupby('class').size())

Finally, we'll split the data into predictor variables and target variable, following by breaking them into train and test sets. We will use 30% of the data as test set.

In [ ]:
Y = data['class'].values
X = data.drop('class', axis=1).values

X_train, X_test, Y_train, Y_test = train_test_split (X, Y, test_size = 0.30, random_state=21)

## Essemble algorithm checking


I evaluate four different ensemble machine learning algorithms, two Boosting and two Bagging methods:
 **Boosting Methods**: AdaBoost (AB) and Gradient Boosting (GBM).
 **Bagging Methods**: Random Forests (RF) and Extra Trees (ET).

I did not standardize the training data (use the data as it is), and do a 10-fold cross-validation for each algorithm.

In [ ]:
# ensembles
ensembles = []
ensembles.append(('AB', AdaBoostClassifier()))
ensembles.append(('GBM', GradientBoostingClassifier()))
ensembles.append(('RF', RandomForestClassifier()))
ensembles.append(('ET', ExtraTreesClassifier()))

In [ ]:
import warnings

results = []
names = []
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for name, model in ensembles:
        kfold = KFold(n_splits=10, random_state=21)
        cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)

Interestingly, all the algorithms hit 100% accuracy in the test! It would be interesting to observe the performance in detail. But for now, I will just pick RandomForestClassifier to validate with the test set.

In [ ]:
# prepare the model
model = RandomForestClassifier(random_state=21, n_estimators=100) 
model.fit(X_train, Y_train)

In [ ]:
predictions = model.predict(X_test)
print("Accuracy score %f" % accuracy_score(Y_test, predictions))
print(classification_report(Y_test, predictions))

In [ ]:
print(confusion_matrix(Y_test, predictions))

## Conclusion

It looks like the essemble method is able to achieve 100% accuracy for this data set. From the confusion matrix shown, there is no single misclassification observed. Personally, I am a little skeptical of this analysis. I will want to investigate this further. Meanwhile, let me know if you have any comments :)